# G2Engine Guide - Why Entities

More information:

1. [GitHub repository](https://github.com/Senzing/docker-jupyter)
1. [Senzing documentation](http://docs.senzing.com/?python#g2config)
1. [G2Engine Reference](senzing-G2Engine-reference.ipynb)

## Prepare environment 

In [ ]:
import com.senzing.g2.engine.G2Engine;
import com.senzing.g2.engine.G2JNI;
import com.senzing.g2.engine.Result;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
public static void RenderJSON(Object obj){
    String str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Run [senzing-G2ConfigMgr-reference.ipynb](senzing-G2ConfigMgr-reference.ipynb)
to install a Senzing Engine configuration in the database.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
RenderJSON(senzingConfigJson);

## G2Engine

In [ ]:
G2Engine g2engine = new G2JNI();

String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int returnCode = g2engine.initV2(moduleName, senzingConfigJson, verboseLogging);
System.out.print(returnCode);

In [ ]:
int returnCode = g2engine.purgeRepository();
System.out.print(returnCode);

### Insert records

Details at [G2Engine.addRecord](senzing-G2Engine-reference.ipynb#addRecord).

#### Insert record 1

In [ ]:
String dataSourceCode = "TEST";
String recordID = "1";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Bob\", \"NAME_LAST\": \"Max\", \"PHONE_NUMBER\": \"123-456-7899\"}";
String loadID = null;

int returnCode = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
System.out.print(returnCode);

#### Insert record 2

In [ ]:
String dataSourceCode = "TEST";
String recordID = "2";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Bob\", \"NAME_LAST\": \"Max\", \"ADDR_FULL\": \"456 Funny ST\"}";
String loadID = null;

int returnCode = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
System.out.print(returnCode);

#### Insert record 3

In [ ]:
String dataSourceCode = "TEST";
String recordID = "2";
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Bob\", \"NAME_LAST\": \"Max\", \"ADDR_FULL\": \"456 Funny ST\", \"PHONE_NUMBER\": \"123-456-7899\"}";
String loadID = null;

int returnCode = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);
System.out.print(returnCode);

### Ask Why

Details at [G2Engine.whyEntityByRecordID](senzing-G2Engine-reference.ipynb#whyEntityByRecordID).

In [ ]:
String datasource_code = "TEST";
String recordID = "2";
StringBuffer response = new StringBuffer();
int returncode = g2engine.whyEntityByRecordID(datasource_code,recordID, response);
RenderJSON(response);